#### Packages

In [1015]:
from gurobipy import *
import pandas as pd
import numpy as np
import grblogtools as glt

#### Read Data

In [1016]:
store_info = pd.read_excel('OR_hw01_data.xlsx', 'Store')
stores = range(len(store_info['Store ID']))
expected_daily_demand = store_info['Expected Daily Demand (unit)']
store_xcoord = store_info['x-coordinate (km)']
store_ycoord = store_info['y-coordinate (km)']

dc_info = pd.read_excel('OR_hw01_data.xlsx', 'DC')
dcs = range(len(dc_info['Location ID']))
maintenance_costs = dc_info['Maintenance Cost ($/unit)']
construction_costs = dc_info['Construction Cost ($)']
dc_xcoord = dc_info['x-coordinate (km)']
dc_ycoord = dc_info['y-coordinate (km)']
dc_maxscale = dc_info['Maximum Scale (unit)']

S = 1 # replenishment cost

#### Model 1(a): min TC (Multiple sourcing)

In [1017]:
eg1a = Model("eg1a")

#-------- Add variables as a list ---------#
# vj = 1 if a DC is built at loc j
v = []
for j in dcs:
    v.append(eg1a.addVar(lb=0, vtype = GRB.BINARY, name = "v" + str(j+1)))

# rij = the amount of products replenished by DCj to store i
r = []
for i in stores:
    r.append([])
    for j in dcs:
        r[i].append(eg1a.addVar(lb = 0, vtype = GRB.INTEGER, name = "r" + str(i+1) + str(j+1)))
        
# Manhattan distancebetween store i and DCj on x-axis
wx = []
for i in stores:
    wx.append([])
    for j in dcs:
        wx[i].append(eg1a.addVar(lb = 0, vtype = GRB.INTEGER, name = "wx" + str(i+1) + str(j+1)))
# Manhattan distancebetween store i and DCj on y-axis
wy = []
for i in stores:
    wy.append([])
    for j in dcs:
        wy[i].append(eg1a.addVar(lb = 0, vtype = GRB.INTEGER, name = "wy" + str(i+1) + str(j+1)))

##### Objective Fucntion

In [1018]:
eg1a.setObjective(
    quicksum(v[j] * construction_costs[j] for j in dcs) \
    + quicksum(maintenance_costs[j] * quicksum(r[i][j] for i in stores) for j in dcs) \
    + quicksum(quicksum(S * r[i][j] * (wx[i][j] + wy[i][j]) for j in dcs) for i in stores)\
    ,GRB.MINIMIZE
)

##### Constraints

In [1019]:
eg1a.addConstrs((quicksum(r[i][j] for j in dcs) == expected_daily_demand[i] for i in stores), "demand_fulfillment")
eg1a.addConstrs((quicksum(r[i][j] for i in stores) <= dc_maxscale[j] for j in dcs), "dcCapacity")
eg1a.addConstrs((quicksum(r[i][j] for i in stores) <= v[j] * 20000 for j in dcs), "replenishment amount exists when vj=1")
eg1a.addConstrs((wx[i][j] >= dc_xcoord[j] - store_xcoord[i] for i in stores for j in dcs), "dis(dc-store), x-axis")
eg1a.addConstrs((wx[i][j] >= store_xcoord[i] - dc_xcoord[j] for j in dcs for i in stores), "dis(store-dc), x-axis")
eg1a.addConstrs((wy[i][j] >= dc_ycoord[j] - store_ycoord[i] for i in stores for j in dcs), "dis(dc-store), y-axis")
eg1a.addConstrs((wy[i][j] >= store_ycoord[i] - dc_ycoord[j] for j in dcs for i in stores), "dis(store-dc), y-axis")

{(0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 4): <gurobi.Constr *Awaiting Model Update*>,
 (0, 5): <gurobi.Constr *Awaiting Model Update*>,
 (0, 6): <gurobi.Constr *Awaiting Model Update*>,
 (0, 7): <gurobi.Constr *Awaiting Model Update*>,
 (0, 8): <gurobi.Constr *Awaiting Model Update*>,
 (0, 9): <gurobi.Constr *Awaiting Model Update*>,
 (0, 10): <gurobi.Constr *Awaiting Model Update*>,
 (0, 11): <gurobi.Constr *Awaiting Model Update*>,
 (0, 12): <gurobi.Constr *Awaiting Model Update*>,
 (0, 13): <gurobi.Constr *Awaiting Model Update*>,
 (0, 14): <gurobi.Constr *Awaiting Model Update*>,
 (0, 15): <gurobi.Constr *Awaiting Model Update*>,
 (0, 16): <gurobi.Constr *Awaiting Model Update*>,
 (0, 17): <gurobi.Constr *Awaiting Model Update*>,
 (0, 18): <gurobi.Constr *Awaiting Model Update*>,
 (0, 19): <gurobi.Constr *Awaiting Model 

##### Optimize

In [1020]:
eg1a.optimize()

In [1021]:
print("1(a) Results | Multiple sourcing: ")

for j in dcs:
    print(v[j].varName, '=', v[j].x)

for j in dcs:
    print("DC" + str(j+1), "\t", end="")
    for i in stores:
        if len(str(r[i][j].x)) < 11:
            print(r[i][j].x, "\t", end="")
        else:
            print(r[i][j].x, "", end="")
    print("")

print("DC's scale: ")
for j in dcs:
    count = 0
    print("DC" + str(j+1), "\t", end="")
    for i in stores:
        count += int(r[i][j].x)
    print(count)

print("z* = ", eg1a.ObjVal)

1(a) Results | Multiple sourcing: 
v1 = 1.0
v2 = 1.0
v3 = 1.0
v4 = 1.0
v5 = 1.0
v6 = 1.0
v7 = 1.0
v8 = -0.0
v9 = 1.0
v10 = 1.0
DC1 	-0.0 	154.0 	-0.0 	-0.0 	-0.0 	-0.0 	-0.0 	-0.0 	-0.0 	144.0 	-0.0 	-0.0 	-0.0 	-0.0 	-0.0 	130.0 	-0.0 	-0.0 	-0.0 	-0.0 	-0.0 	132.0 	0.0 	-0.0 	-0.0 	-0.0 	-0.0 	-0.0 	-0.0 	-0.0 	114.0 	-0.0 	107.0 	-0.0 	184.0 	-0.0 	-0.0 	-0.0 	-0.0 	-0.0 	-0.0 	-0.0 	-0.0 	-0.0 	-0.0 	-0.0 	-0.0 	0.0 	-0.0 	-0.0 	-0.0 	-0.0 	-0.0 	0.0 	137.0 	-0.0 	-0.0 	149.0 	-0.0 	-0.0 	-0.0 	-0.0 	154.0 	131.0 	178.0 	152.0 	-0.0 	-0.0 	-0.0 	-0.0 	0.0 	-0.0 	152.0 	-0.0 	-0.0 	-0.0 	-0.0 	122.0 	-0.0 	189.0 	109.0 	-0.0 	-0.0 	-0.0 	-0.0 	77.0 	-0.0 	-0.0 	-0.0 	-0.0 	-0.0 	191.0 	-0.0 	-0.0 	178.0 	-0.0 	122.0 	-0.0 	-0.0 	118.0 	
DC2 	-0.0 	-0.0 	-0.0 	-0.0 	-0.0 	-0.0 	-0.0 	-0.0 	-0.0 	-0.0 	-0.0 	-0.0 	-0.0 	-0.0 	-0.0 	-0.0 	-0.0 	-0.0 	-0.0 	-0.0 	-0.0 	-0.0 	-0.0 	-0.0 	-0.0 	-0.0 	-0.0 	-0.0 	-0.0 	200.0 	-0.0 	-0.0 	0.0 	-0.0 	-0.0 	-0.0 	-0.0 	-0.0 	-0.0 	-0.0 	-0.0 

#### Model 1(b): min TC (Single sourcing)

In [1022]:
eg1b = Model("eg1b")
eg1b.Params.LogFile = "eg1b.log" 
results = glt.parse("eg1b*.log")
summary = results.summary()
nodelog_progress = results.progress("nodelog")


#-------- Add variables as a list ---------#
# vj = 1 if a DC is built at loc j
v = []
for j in dcs:
    v.append(eg1b.addVar(lb=0, vtype = GRB.BINARY, name = "v" + str(j+1)))

# rij = the amount of products replenished by DCj to store i
r = []
for i in stores:
    r.append([])
    for j in dcs:
        r[i].append(eg1b.addVar(lb = 0, vtype = GRB.INTEGER, name = "r" + str(i+1) + str(j+1)))
        
# zij = 1 if products are replenished by DCj to store i (rij>0)
z = []
for i in stores:
    z.append([])
    for j in dcs:
        z[i].append(eg1b.addVar(lb = 0, vtype = GRB.BINARY, name = "z" + str(i+1) + str(j+1)))

# Manhattan distancebetween store i and DCj on x-axis
wx = []
for i in stores:
    wx.append([])
    for j in dcs:
        wx[i].append(eg1b.addVar(lb = 0, vtype = GRB.INTEGER, name = "wx" + str(i+1) + str(j+1)))
# Manhattan distancebetween store i and DCj on y-axis
wy = []
for i in stores:
    wy.append([])
    for j in dcs:
        wy[i].append(eg1b.addVar(lb = 0, vtype = GRB.INTEGER, name = "wy" + str(i+1) + str(j+1)))

##### Objective Function

In [1023]:
eg1b.setObjective(
    quicksum(v[j] * construction_costs[j] for j in dcs) \
    + quicksum(v[j] * maintenance_costs[j] * quicksum(r[i][j] for i in stores) for j in dcs) \
    + quicksum(quicksum(r[i][j] * (wx[i][j] + wy[i][j]) for i in stores) for j in dcs)\
    ,GRB.MINIMIZE
)

##### Constraints

In [1024]:
eg1b.addConstrs((quicksum(v[j] * r[i][j] for j in dcs) >= expected_daily_demand[i] for i in stores), "demand_fulfillment")
eg1b.addConstrs((quicksum(z[i][j] for j in dcs) == 1 for i in stores), "single sourcing1")
eg1b.addConstrs((quicksum(z[i][j] for j in dcs) <= v[j] for i in stores), "single sourcing2")
eg1b.addConstrs((quicksum(v[j] * r[i][j] for i in stores) <= dc_maxscale[j] for j in dcs), "dcCapacity")
eg1b.addConstrs((quicksum(v[j] * r[i][j] for i in stores) >= 0 for j in dcs), "replenishment amount >= 0")
eg1b.addConstrs((wx[i][j] >= dc_xcoord[j] - store_xcoord[i] for i in stores for j in dcs), "dis(dc-store), x-axis")
eg1b.addConstrs((wx[i][j] >= store_xcoord[i] - dc_xcoord[j] for j in dcs for i in stores), "dis(store-dc), x-axis")
eg1b.addConstrs((wy[i][j] >= dc_ycoord[j] - store_ycoord[i] for i in stores for j in dcs), "dis(dc-store), y-axis")
eg1b.addConstrs((wy[i][j] >= store_ycoord[i] - dc_ycoord[j] for j in dcs for i in stores), "dis(store-dc), y-axis")

{(0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 4): <gurobi.Constr *Awaiting Model Update*>,
 (0, 5): <gurobi.Constr *Awaiting Model Update*>,
 (0, 6): <gurobi.Constr *Awaiting Model Update*>,
 (0, 7): <gurobi.Constr *Awaiting Model Update*>,
 (0, 8): <gurobi.Constr *Awaiting Model Update*>,
 (0, 9): <gurobi.Constr *Awaiting Model Update*>,
 (0, 10): <gurobi.Constr *Awaiting Model Update*>,
 (0, 11): <gurobi.Constr *Awaiting Model Update*>,
 (0, 12): <gurobi.Constr *Awaiting Model Update*>,
 (0, 13): <gurobi.Constr *Awaiting Model Update*>,
 (0, 14): <gurobi.Constr *Awaiting Model Update*>,
 (0, 15): <gurobi.Constr *Awaiting Model Update*>,
 (0, 16): <gurobi.Constr *Awaiting Model Update*>,
 (0, 17): <gurobi.Constr *Awaiting Model Update*>,
 (0, 18): <gurobi.Constr *Awaiting Model Update*>,
 (0, 19): <gurobi.Constr *Awaiting Model 

##### Optimize

In [1025]:
eg1b.optimize()

##### Results

In [1026]:
print("1(b) Results | Single sourcing: ")

for j in dcs:
    print(v[j].varName, '=', v[j].x)

replenish_plan = []   
for j in dcs:
    print("DC" + str(j+1), "\t", end="")
    for i in stores:
        if len(str(r[i][j].x)) < 11:
            print(r[i][j].x, "\t", end="")
        else:
            print(r[i][j].x, "", end="")
    print("")

print("DC's scale: ")
for j in dcs:
    count = 0
    print("DC" + str(j+1), "\t", end="")
    for i in stores:
        count += int(r[i][j].x)
    print(count)

print("z* = ", eg1b.ObjVal)

1(b) Results | Single sourcing: 
v1 = 1.0
v2 = 1.0
v3 = 1.0
v4 = 1.0
v5 = 1.0
v6 = 1.0
v7 = 1.0
v8 = 0.0
v9 = 1.0
v10 = 1.0
DC1 	0.0 	154.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	144.0 	0.0 	0.0 	0.0 	0.0 	0.0 	130.0 	0.0 	0.0 	0.0 	0.0 	0.0 	132.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	114.0 	0.0 	107.0 	0.0 	184.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	137.0 	0.0 	0.0 	149.0 	0.0 	0.0 	0.0 	0.0 	154.0 	131.0 	178.0 	152.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	152.0 	0.0 	0.0 	0.0 	0.0 	122.0 	0.0 	189.0 	109.0 	0.0 	0.0 	0.0 	0.0 	77.0 	0.0 	0.0 	0.0 	0.0 	0.0 	191.0 	0.0 	0.0 	178.0 	0.0 	122.0 	0.0 	0.0 	118.0 	
DC2 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	200.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	132.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	166.0 	0.0 	0.0 	0.0 	0

In [1027]:
glt.plot(results.summary())

interactive(children=(Dropdown(description='x', options=('Platform', 'Time', 'PhysicalCores', 'LogicalProcesso…

#### Model 2: min the maximum distance

In [1028]:
eg2 = Model("eg2")
B = int #cost_constraint

#-------- Add variables as a list ---------#

# vj = 1 if a DC is built at loc j
v = []
for j in dcs:
    v.append(eg2.addVar(lb=0, vtype = GRB.BINARY, name = "v" + str(j+1)))

# rij = the amount of products replenished by DCj to store i
r = []
for i in stores:
    r.append([])
    for j in dcs:
        r[i].append(eg2.addVar(lb = 0, vtype = GRB.INTEGER, name = "r" + str(i+1) + str(j+1)))
        
# zij = 1 if products are replenished by DCj to store i (rij>0)
z = []
for i in stores:
    z.append([])
    for j in dcs:
        z[i].append(eg2.addVar(lb = 0, vtype = GRB.BINARY, name = "z" + str(i+1) + str(j+1)))

# Manhattan distancebetween store i and DCj on x-axis
wx = []
for i in stores:
    wx.append([])
    for j in dcs:
        wx[i].append(eg2.addVar(lb = 0, vtype = GRB.INTEGER, name = "wx" + str(i+1) + str(j+1)))
# Manhattan distancebetween store i and DCj on y-axis
wy = []
for i in stores:
    wy.append([])
    for j in dcs:
        wy[i].append(eg2.addVar(lb = 0, vtype = GRB.INTEGER, name = "wy" + str(i+1) + str(j+1)))


# wij=1 if the DCj is the closest to store i
w = []
for i in stores:
    w.append([])
    for j in dcs:
        w[i].append(eg2.addVar(lb = 0, vtype = GRB.BINARY, name = "w" + str(i+1) + str(j+1)))

d = eg2.addVar(lb = 0, vtype = GRB.INTEGER, name = "d")
B = eg2.addVar(lb = 0, vtype = GRB.INTEGER, name = "B")

##### Objective Function

In [1029]:
eg2.setObjective(d,GRB.MINIMIZE)

##### Constraints

In [1030]:
eg2.addConstrs((quicksum(v[j] * r[i][j] for j in dcs) >= expected_daily_demand[i] for i in stores), "demand_fulfillment")
eg2.addConstrs((quicksum(v[j] * r[i][j] for i in stores) <= dc_maxscale[j] for j in dcs), "dcCapacity")
#eg2.addConstrs((quicksum(construction_costs[j] + maintenance_costs[j] * r[i][j] for i in stores)for j in dcs) <= B, "dcCapacity")
eg2.addConstrs((quicksum(r[i][j] for i in stores) >= 0 for j in dcs), "replenishment amount >= 0")
eg2.addConstrs((quicksum(w[i][j] for j in dcs) == 1 for i in stores), "每間store都有一個最近的DC")
eg2.addConstrs((w[i][j] <= v[j] for i in stores for j in dcs), "要確定那間DC有蓋")
eg2.addConstrs((d >= quicksum(w[i][j] * (wx[i][j] + wy[i][j]) for j in dcs) for i in stores), "每間store到最近的DC的最遠距離")
eg2.addConstrs((wx[i][j] >= dc_xcoord[j] - store_xcoord[i] for i in stores for j in dcs), "dis(dc-store), x-axis")
eg2.addConstrs((wx[i][j] >= store_xcoord[i] - dc_xcoord[j] for j in dcs for i in stores), "dis(store-dc), x-axis")
eg2.addConstrs((wy[i][j] >= dc_ycoord[j] - store_ycoord[i] for i in stores for j in dcs), "dis(dc-store), y-axis")
eg2.addConstrs((wy[i][j] >= store_ycoord[i] - dc_ycoord[j] for j in dcs for i in stores), "dis(store-dc), y-axis")

{(0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 4): <gurobi.Constr *Awaiting Model Update*>,
 (0, 5): <gurobi.Constr *Awaiting Model Update*>,
 (0, 6): <gurobi.Constr *Awaiting Model Update*>,
 (0, 7): <gurobi.Constr *Awaiting Model Update*>,
 (0, 8): <gurobi.Constr *Awaiting Model Update*>,
 (0, 9): <gurobi.Constr *Awaiting Model Update*>,
 (0, 10): <gurobi.Constr *Awaiting Model Update*>,
 (0, 11): <gurobi.Constr *Awaiting Model Update*>,
 (0, 12): <gurobi.Constr *Awaiting Model Update*>,
 (0, 13): <gurobi.Constr *Awaiting Model Update*>,
 (0, 14): <gurobi.Constr *Awaiting Model Update*>,
 (0, 15): <gurobi.Constr *Awaiting Model Update*>,
 (0, 16): <gurobi.Constr *Awaiting Model Update*>,
 (0, 17): <gurobi.Constr *Awaiting Model Update*>,
 (0, 18): <gurobi.Constr *Awaiting Model Update*>,
 (0, 19): <gurobi.Constr *Awaiting Model 

##### Optimize

In [1031]:
eg2.optimize()

##### Results

In [1032]:
print("3 Results | Multiple sourcing: ")

for j in dcs:
    print(v[j].varName, '=', v[j].x)

replenish_plan = []   
for j in dcs:
    print("DC" + str(j+1), "\t", end="")
    for i in stores:
        if len(str(r[i][j].x)) < 11:
            print(r[i][j].x, "\t", end="")
        else:
            print(r[i][j].x, "", end="")
    print("")

print("DC's scale: ")
for j in dcs:
    count = 0
    print("DC" + str(j+1), "\t", end="")
    for i in stores:
        count += int(r[i][j].x)
    print(count)

print("z* = ", eg2.ObjVal)

3 Results | Multiple sourcing: 
v1 = 1.0
v2 = 1.0
v3 = 1.0
v4 = 1.0
v5 = 1.0
v6 = 1.0
v7 = 1.0
v8 = 1.0
v9 = 1.0
v10 = 1.0
DC1 	0.0 	154.0 	115.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	144.0 	0.0 	135.0 	161.0 	0.0 	142.0 	130.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	160.0 	0.0 	0.0 	0.0 	0.0 	114.0 	0.0 	0.0 	114.0 	164.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	150.0 	0.0 	0.0 	135.0 	0.0 	0.0 	164.0 	135.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	143.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	157.0 	0.0 	0.0 	154.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	122.0 	0.0 	0.0 	0.0 	0.0 	0.0 	106.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	103.0 	104.0 	0.0 	0.0 	0.0 	0.0 	0.0 	118.0 	
DC2 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	178.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	132.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	171.0 	0.0 	0.0 	184.0 	0.0 	0.0 	0.0 	0.0 	0.

#### Visualization